In [1]:
from sympy import *
from scipy.integrate import quad
x = symbols('x')
import math

In [2]:
def imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar):
    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1))

    Bfunvec1_prime = quad(tempFunction, 0, j0_prime, args=(theta, sigmastar))[0]
    Bfunvec2_prime = quad(tempFunction, 0, jxbar_prime, args=(theta, sigmastar))[0]

    return (Bfunvec1_prime, Bfunvec2_prime)

In [59]:
Cex = 0.421601
Ceystar = 26.059874
Cey = 4.598402945
Cem = 1.196110986
jxbar = Cex/(Cex + Ceystar)
jmbar = Cey / (Cey + Cem)
alpha = 0.85
theta = 4
## phi = 5
pe = 1.004700499
te = 0.368354551
tb = 0.349449338
sigmastar = 1
sigma = 1
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
varphi = 1
prop = 0.449691508

Vg_prime = 38.63009333
Vgstar_prime = 176.5431669

In [60]:
def g(p, alpha = 0.15):
    return alpha**(-alpha) * (1-alpha)**(-(1-alpha)) * p**alpha

def gprime(p, alpha = 0.15):
    return (alpha/(1-alpha))**(1-alpha) * p**(-(1-alpha))

def k(p):
    return gprime(p) / (g(p)-p*gprime(p))

def gdprime(p, alpha = 0.15):
    return -alpha * p**(alpha - 2) * ((1-alpha)/alpha)**alpha

def Dstar(p, sigmastar):
    return gprime(p) * g(p)**(-sigmastar)

def Dstarprime(p, sigmastar):
    return diff(Dstar(x, sigmastar),x).subs(x,p)

def D1star(p,t,sigmastar):
    return g(p)**(-sigmastar) * gprime(p + t)

def D1starprime(p,t,sigmastar):
    return diff(D1star(x,t,sigmastar),x).subs(x,p)

In [61]:
def jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe) + tb * gprime(pe))**(-theta) * Ceystar
    return num / denum

def j0_new(pe,tb,jxbar,theta,Cex,Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe))**(-theta) * Ceystar
    return num / denum

def Ceystar_new(pe,tb, jxbar_p, jxbar, sigmastar, theta, Ceystar):
    const = gprime(pe) * g(pe)**(-sigmastar) * Ceystar
    frac = ((1-jxbar_p)/ (1-jxbar))**(1+(1-sigmastar)/theta)
    #frac = ((1-jxbar_new(pe,tb,jxbar,theta,Cex,Ceystar))/ (1-jxbar))**(1+(1-sigmastar)/theta)
    return const * frac / (g(1)**(-sigmastar) * gprime(1))

def Cey_new(pe,tb, sigmastar, theta, Cey):
    const = gprime(pe + tb) * g(pe+tb)**(-sigma) * Cey
    return const / (g(1)**(-sigma) * gprime(1))

def Cex1_new(pe,tb, j0bar_p, j0bar, sigmastar, Cex):
    const = gprime(pe + tb) * g(pe+tb)**(-sigmastar) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = (j0bar_p / j0bar) ** (1+ (1-sigmastar)/theta)
    return const * frac

def Cex2_new(pe,tb,jxbar_p, j0bar_p, jxbar, sigmastar, theta):
    const = g(pe)**(-sigmastar) * gprime(pe + tb) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = ((1-jxbar)/jxbar)**(sigmastar/theta) * (theta + 1 - sigmastar)/theta
    jterm = 1/ jxbar**(1+(1-sigmastar)/theta)
    B1, B2 = imcomp_betas(j0bar_p, jxbar_p, theta, sigmastar)
    return const * frac * jterm * (B2- B1)
    

def Cem_new(pe, tb, sigma, Cem):
    const = gprime(pe + tb) * g(pe + tb)**(-sigma) / (g(1)**(-sigmastar) * gprime(1)) * Cem
    return const
    
def Vgx2_new(pe, jxbar_p, jxbar, j0bar_p, Cex, theta, sigmastar):
    pterm = g(pe)**(1-sigmastar) *g(1)* Cex / (gprime(1) * g(1)**(1-sigmastar))
    num = (1-j0bar_p)**((theta + 1 - sigmastar)/theta) - (1-jxbar_p)**((theta + 1 - sigmastar) / theta)
    denum = jxbar * (1-jxbar)**((1-sigmastar)/theta)
    return pterm * num / denum

def Vgx1_new(pe, tb, j0_p, jxbar, Cex, theta, sigmastar):
    const = (g(pe+tb) / g(1))**(1-sigmastar) * (j0_p / jxbar)**(1+(1-sigmastar)/theta) * g(1)/gprime(1) * Cex
    return const

def S_new(pe, tb, Cex2_prime, Vgx2_prime):
    const = g(pe+tb) / gprime(pe+tb) * Cex2_prime - Vgx2_prime
    return const

def Vgm_new(pe, tb, Cem):
    return g(pe+tb)**(1-sigma) * Cem * g(1) / (g(1)**(1-sigma) * gprime(1))

def Lg_new(pe, tb, Cey_prime, Cex1_prime, Cex2_prime):
    return (1/k(pe+tb)) * (Cey_prime + Cex1_prime + Cex2_prime)

def Lgstar_new(pe,tb,Ceystar_prime, Cem_prime):
    return 1/k(pe) * Ceystar_prime + 1/k(pe+tb) * Cem_prime

def Vgstar_new(pe,jxbar_prime,jxbar, sigmastar,theta,Ceystar, Vgx1_prime, Vgx2_prime):
    Vgystar_prime = (g(pe) / g(1))**(1-sigmastar) * ((1-jxbar_prime)/(1-jxbar)) ** (1+(1-sigmastar)/theta) * g(1)/ gprime(1) * Ceystar
    return Vgystar_prime + Vgx1_prime + Vgx2_prime


In [62]:
jxbar_prime = jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar)
#jxbar_prime = jxbar
j0_prime = j0_new(pe,tb,jxbar,theta,Cex,Ceystar)
Cey_prime = Cey_new(pe,tb, sigmastar, theta, Cey)
Cex1_prime = Cex1_new(pe,tb, j0_prime, jxbar, sigmastar, Cex)
Cex2_prime = Cex2_new(pe, tb, jxbar_prime, j0_prime, jxbar, sigmastar, theta)
Cem_prime = Cem_new(pe, tb, sigma, Cem)
Ceystar_prime = Ceystar_new(pe,tb, jxbar_prime, jxbar, sigmastar, theta, Ceystar)
Vgx1_prime = Vgx1_new(pe, tb, j0_prime, jxbar, Cex, theta, sigmastar)
Vgx2_prime = Vgx2_new(pe, jxbar_prime, jxbar, j0_prime, Cex, theta, sigmastar)
Vgm_prime = Vgm_new(pe, tb, Cem)
S = S_new(pe, tb, Cex2_prime, Vgx2_prime)
Lg_prime = Lg_new(pe, tb, Cey_prime, Cex1_prime, Cex2_prime)
Lgstar_prime =Lgstar_new(pe,tb,Ceystar_prime, Cem_prime)
Vgstar_prime = Vgstar_new(pe,jxbar_prime,jxbar, sigmastar,theta,Ceystar, Vgx1_prime, Vgx2_prime)
sigmastar = 1

In [63]:
(g(pe) / g(1)) ** (1-sigmastar) * ((1-jxbar_prime) / (1-jxbar)) ** (1+(1-sigmastar) / theta)

0.9996079502951188

In [64]:
(g(pe+tb) / g(1))**(1-sigmastar) * (j0_prime / jxbar) ** (1+(1-sigmastar) / theta) + (g(pe) / g(1)) ** (1-sigmastar) * ((1-j0_prime) ** (1+(1-sigmastar) / theta) - (1-jxbar_prime) ** (1+(1-sigmastar) / theta)) / jxbar / (1-jxbar) ** ((1-sigmastar)/theta)

1.0242332582487654

In [65]:
(g(pe+tb) / g(1))**(1-sigmastar) * (j0_prime / jxbar) ** (1+(1-sigmastar) / theta)

0.8382183137006431

In [66]:
(g(pe) / g(1)) ** (1-sigmastar) * ((1-j0_prime) ** (1+(1-sigmastar) / theta) - (1-jxbar_prime) ** (1+(1-sigmastar) / theta)) / jxbar /(1-jxbar) ** ((1-sigmastar)/theta)

0.18601494454812229

In [67]:
(g(pe) / g(1)) ** (1-sigmastar) * ((1-j0_prime) ** (1+(1-sigmastar) / theta) - (1-jxbar_prime) ** (1+(1-sigmastar) / theta)) / jxbar /(1-jxbar) ** ((1-sigmastar)/theta) * Cex * g(1) / gprime(1)

0.522827244242886

In [68]:
Vgx2_prime

0.522827244242886

In [7]:
Cex_prime = gprime(pe) * g(pe)**(-sigmastar) / (g(1)**(-sigmastar) * gprime(1)) * (jxbar_prime / jxbar) ** (1+ (1-sigmastar)/theta) * Cex
Ce_prime = Cey_prime + Cem_prime
Cestar_prime = Ceystar_prime + Cex_prime
Ceworld_prime = Ce_prime + Cestar_prime
Qe_prime =4.40505495
Qestar_prime = 27.33067971
Qeworld_prime = Qe_prime + Qestar_prime


In [8]:
dcestardpe = abs(Dstarprime(pe,sigmastar) / Dstar(pe,sigmastar) * Cex_prime 
                      #+ D1starprime(pe,tb,sigmastar) / D1star(pe,tb,sigmastar) * Cex2_prime
                      + Dstarprime(pe,sigmastar) / Dstar(pe,sigmastar) * (Ceystar_prime))
numerator = varphi * (epsilonS * Qe_prime + epsilonSstar * Qestar_prime)
denominator = numerator / varphi + dcestardpe * pe

In [9]:
epsilonSw = (Qe_prime) * epsilonS / Qeworld_prime + Qestar_prime * epsilonSstar / Qeworld_prime
numerator = varphi * epsilonSw * Qeworld_prime
denominator = epsilonSw * Qeworld_prime + epsilonDstar * Cestar_prime

In [10]:
rho = 0.0001

In [11]:
def g1(p, rho = rho):
    alpha = 0.15
    t1 = (1-alpha)**(1/(1-rho))
    t2 = alpha**(1/(1-rho)) * p**(-rho/(1-rho))
    return (t1 + t2)**(-(1-rho)/rho)

def g1prime(p, rho = rho):
    alpha = 0.15
    return diff(g1(x, rho),x).subs(x,p)

In [12]:
def cex_new(pe,ve, jxbar, Cex, Ceystar, sigmastar, theta):
    jxbar_p = jxbar_temp(pe,ve, theta, Cex, Ceystar)
    return Dstar(ve, sigmastar) / Dstar(1, sigmastar) * (jxbar_p / jxbar) ** (1+(1-sigmastar) / theta) * Cex

def ceystar_new(pe,ve, jxbar, Cex, Ceystar, sigmastar, theta):
    jxbar_p = jxbar_temp(pe,ve, theta, Cex, Ceystar)
    return Dstar(pe, sigmastar) / Dstar(1, sigmastar) * ((1-jxbar_p) / (1-jxbar)) ** (1+(1-sigmastar) / theta) * Ceystar

def cey_new(pe,ve,jmbar, Cey, Cem, sigma, theta):
    jmbar_p = jmbar_temp(pe,ve, theta, Cey, Cem)
    return Dstar(ve, sigma) / Dstar(1,sigma) * (jmbar_p / jmbar)**(1+(1-sigma)/theta) * Cey

def cem_new(pe,ve,jmbar, Cey, Cem, sigma, theta):
    jmbar_p = jmbar_temp(pe,ve, theta, Cey, Cem)
    return Dstar(pe, sigma) / Dstar(1,sigma) * ((1-jmbar_p) / (1-jmbar))**(1+(1-sigma)/theta) * Cem

def jxbar_temp(pe,ve, theta, Cex, Ceystar):
    return Cex / (Cex + (g(ve) / g(pe)) ** theta * Ceystar)

def jmbar_temp(pe,ve, theta, Cey, Cem):
    return Cey / (Cey + Cem * (g(ve) / g(pe))**theta)



In [14]:
## PC hybrid
pe = 0.965289922
tb = 0.368354551
prop = 0.449691508
re = (pe + tb) / pe
ve = (pe + tb - prop*tb)
jmbar_prime =0.767046309
jxbar_prime = 0.014224486
Cey_prime = 3.447997017
Cex_prime = 0.32250493
Cem_prime = 0.896873797
Ceystar_prime = 27.04347061
Qe_prime = 4.40160679
Qestar_prime = 27.30928598
epsilonSw = (Qe_prime) * epsilonS / Qeworld_prime + Qestar_prime * epsilonSstar / Qeworld_prime
djxbardpe = theta * gprime(pe) / g(pe) * jxbar_prime * (1-jxbar_prime)
djxbardpe = diff(jxbar_temp(x,ve,theta,Cex,Ceystar),x).subs(x,pe)
djmbardpe = theta * gprime(pe) / g(pe) * jmbar_prime * (1-jmbar_prime)

In [18]:
## pure tp
pe = 0.96549695
tb = 0.533349171
ve = (pe + tb)
Cey_prime = 2.887942953
Cex_prime = 0.21684408
Cem_prime = 1.518318529
Ceystar_prime = 27.09118773
Qe_prime = 4.402078775
Qestar_prime = 27.31221436
jxbar_prime = 0.012273331
jmbar_prime = 0.747013829
epsilonSw = (Qe_prime) * epsilonS / Qeworld_prime + Qestar_prime * epsilonSstar / Qeworld_prime
djxbardpe = theta * gprime(pe) / g(pe) * jxbar_prime * (1-jxbar_prime)
djxbardpe = diff(jxbar_temp(x,ve,theta,Cex,Ceystar),x).subs(x,pe)
djmbardpe = theta * gprime(pe) / g(pe) * jmbar_prime * (1-jmbar_prime)

In [15]:
## new leakage for PC, EPC
djxdve = -jxbar_prime * (1-jxbar_prime) * gprime(ve) / g(ve) * theta
djmdve = -jmbar_prime * (1-jmbar_prime) * gprime(ve) / g(ve) * theta

dceydve = Dstarprime(ve, sigma) / Dstar(ve, sigma) * Cey_prime + (1+(1-sigma)/theta) * djmdve / jmbar_prime * Cey_prime
dcemdve = 1/(1-jmbar_prime) * (1+(1-sigma)/theta) * (-djmdve) * Cem_prime 
dcexdve = Dstarprime(ve, sigmastar) / Dstar(ve, sigmastar) * Cex_prime +  (1+(1-sigmastar) / theta) * Cex_prime / jxbar_prime * djxdve
dceystardve = (1+(1-sigmastar) /theta) * Ceystar_prime * (-djxdve) / (1-jxbar_prime)

leak = -(dceystardve + dcemdve) / (dcexdve + dceydve)
leak2 = -dceystardve / dcexdve

In [16]:
numerator = varphi * epsilonSw * Qeworld_prime
dceystardpe = abs(Dstarprime(pe, sigmastar) / Dstar(pe, sigmastar) - (1+(1-sigmastar) / theta) / (1-jxbar_prime) * djxbardpe) * Ceystar_prime
#dceystardpe = abs(Dstarprime(pe, sigmastar) / Dstar(pe, sigmastar) * Ceystar_prime - Dstar(pe,sigmastar) * djxbardpe)
dcexdpe = abs((1+(1-sigmastar) / theta) / (jxbar_prime) * djxbardpe) * Cex_prime

In [17]:
djxbardpe = theta * gprime(pe) / g(pe) * jxbar_prime * (1-jxbar_prime)
djmbardpe = theta * gprime(pe) / g(pe) * jmbar_prime * (1-jmbar_prime)
dceystardpe = abs(Dstarprime(pe, sigmastar) / Dstar(pe, sigmastar) - (1+(1-sigmastar) / theta) / (1-jxbar_prime) * djxbardpe) * Ceystar_prime
dcexdpe = abs((1+(1-sigmastar) / theta) / (jxbar_prime) * djxbardpe) * Cex_prime
dcemdpe = abs(Dstarprime(pe,sigma) / Dstar(pe,sigma) - (1+(1-sigma)/theta) / (1-jmbar_prime) * djmbardpe) * Cem_prime
dceydpe = abs((1+(1-sigma)/theta) / jmbar_prime * djmbardpe) * Cey_prime
(dceystardpe + dcemdpe)* pe - leak * (dcexdpe + dceydpe) *pe

28.4864832360412

In [23]:
Dstarprime(pe, sigmastar) / Dstar(pe, sigmastar) - (1+(1-sigmastar) / theta) / (1-jxbar_prime) * djxbardpe * Ceystar_prime + (1+(1-sigmastar) / theta) / (jxbar_prime) * djxbardpe* Cex_prime

-1.07745577765086

In [19]:
epsilonDstar * (Ceystar_prime + Cem_prime) + leak * epsilonD * (Cex_prime + Cey_prime)

29.3536602474179

In [20]:
diff(cem_new(x,pe+tb,jmbar, Cey, Cem, sigma, theta), x).subs(x,pe)

-2.27741938063412

In [21]:
diff(ceystar_new(x,x+tb, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,pe)

-28.1328465868253

In [22]:
dgestardpe = diff(ceystar_new(x,x+tb, jxbar, Cex, Ceystar, sigmastar, theta) + cem_new(x,x+tb,jmbar, Cey, Cem, sigma, theta), x).subs(x,pe)
dgestardpe

-29.9562347328241

In [23]:
dgedpe = diff(cey_new(x,x+tb,jmbar, Cey, Cem, sigma, theta) + cex_new(x,x+tb, jxbar, Cex, Ceystar, sigmastar, theta), x).subs(x,pe)
dgedpe

-1.86252604944785

In [24]:
dgestardpe * pe + pe* leak * dgedpe

-29.3536602044735

In [25]:
diff(jxbar_temp(x,x+tb, theta, Cex, Ceystar), x).subs(x,pe)

0.00268073643233450

In [26]:
-theta * g(pe) / g(ve) * (gprime(ve) * g(pe) - gprime(pe) * g(ve)) / g(pe)**2 * jxbar_prime * (1-jxbar_prime)

0.002680736621134375

In [156]:
diff(ceystar_new(pe,x, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,ve)

0.133102025229275

In [66]:
abs(diff(ceystar_new(x,x+tb - prop*tb, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,pe)) * pe + pe*prop*abs(diff(cex_new(x,x+tb - prop*tb, jxbar, Cex, Ceystar, sigmastar, theta), x).subs(x,pe))

27.1884985168510

In [166]:
(dgestardpe + leak * dgedpe)

-30.4026441559173

In [168]:
(dceystardpe + dcemdpe)- leak * (dcexdpe + dceydpe)

30.4026442003964

In [38]:
Dstarprime(pe+tb, sigmastar) / Dstar(ve, sigmastar)

-0.667179896581258

In [35]:
diff(jxbar_temp(x,x+tb, theta, Cex, Ceystar), x).subs(x,pe) / jxbar_prime * (1+(1-sigmastar)/theta) * Dstar(pe + tb, sigmastar)

0.0218587780573164

In [40]:
diff(gprime(x),x).subs(x,ve) / gprime(ve) - sigmastar* gprime(ve)/g(ve)

-0.667179896581258

In [41]:
sigmastar* gprime(ve)/g(ve)

0.10007698448718873